In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

# Imports

In [3]:
import jax

# Get the percision of the jax library
print()
print('Jax has been configured to use the following devices: ', jax.devices())
jax.config.update("jax_enable_x64", True)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.



Jax has been configured to use the following devices:  [CpuDevice(id=0)]


In [4]:
import numpy as np
from ICARUS.propulsion.engine import Engine
from ICARUS.database import DB

#  Load Plane and Engine

In [5]:
engine_dir = "../Data/Engine/Motor_1/"

engine = Engine()
engine.load_data_from_df(engine_dir)
# engine.plot_engine_map()

# Vehicle

In [6]:
from ICARUS.vehicle.plane import Airplane
from ICARUS.mission.mission_vehicle import MissionVehicle

In [7]:
plane: Airplane = DB.get_vehicle('final_design')
# plane.visualize(annotate=True)
# plane.main_wing.symmetries

mission_plane = MissionVehicle(
    plane,
    engine,
    solver= "AVL"
)

from ICARUS.visualization.airplane.db_polars import plot_airplane_polars
plot_airplane_polars([plane.name])

Loaded Plane Plane Object: final_design
Run Doesn't Exist: final_design,'GNVP3 Potential CL' 
Run Doesn't Exist: final_design,'GNVP3 2D CL' 
Run Doesn't Exist: final_design,'GNVP7 Potential CL' 
Run Doesn't Exist: final_design,'GNVP7 2D CL' 
Run Doesn't Exist: final_design,'LSPT Potential CL' 
Run Doesn't Exist: final_design,'LSPT 2D CL' 


(array([[<Axes: title={'center': 'CL vs AoA'}, xlabel='AoA', ylabel='CL'>,
         <Axes: title={'center': 'CD vs AoA'}, xlabel='AoA', ylabel='CD'>],
        [<Axes: title={'center': 'Cm vs AoA'}, xlabel='AoA', ylabel='Cm'>,
         <Axes: title={'center': 'CD vs CL'}, xlabel='CL', ylabel='CD'>],
        [<Axes: >, <Axes: >]], dtype=object),
 <Figure size 1000x1000 with 4 Axes>)

# Compute Trajectory

In [ ]:
from ICARUS.geometry.polynomial import h_polynomial_factory
from ICARUS.mission.trajectory.trajectory import MissionTrajectory
from ICARUS.mission.trajectory.integrate import RK45_scipy_integrator
from ICARUS.geometry.cubic_splines import CubicSpline_factory
import jax.numpy as jnp

In [ ]:
operating_floor = 12.5
t0 = 0
x0 =  jnp.array([0., 20.])

# Polynomial Trajectory
coeffs = jnp.array(
    [
        x0[1],
        0.2,
        -1/300
    ]
)
traj_fun_1 = h_polynomial_factory(coeffs)

traj_poly  = MissionTrajectory(
    "Polynomial Trajectory", 
    traj_fun_1, 
    vehicle=mission_plane,
    verbosity= 2,
    operating_floor= 12.4
)

# Spline Trajectory
# y= [20.34201669, 18.79171704 ,20.01380251 ,20.70916841, 20.42523286 ,19.08487198]
y = [20, 20 , 20, 20, 20, 20,20, 20, 20,20, 20, 20]
# y = [20, 20 , 20, 20, 20, 20,]
x =  jnp.linspace(0, 10000, len(y)+2)
y = jnp.array([x0[1], *y, operating_floor])

cub_spl = jax.jit(CubicSpline_factory)

spline_i = cub_spl(x,y)

traj_spl = MissionTrajectory(
    "Spline Trajectory", 
    spline_i, 
    vehicle=mission_plane,
    verbosity= 2,
    operating_floor= 12.4
)
gamma = jnp.arctan(traj_spl.dy_dx(x0[0]))


In [ ]:
traj_spl.get_control(y0[:2], y0[2:])

In [ ]:
x0 =  jnp.array([0., 20.])
g1 = jnp.arctan(
    traj_spl.dy_dx(x0[0])
)
v0_mag = 20.
v0 = jnp.array([jnp.cos(g1), jnp.sin(g1)]) * v0_mag
y0 = jnp.hstack([x0, v0])
print(y0)

traj_spl.timestep(0.1, y0)
state = traj_spl.get_control()
traj_spl.dvdt(
    0.1,
    y0[:2],
    y0[2:],
    state
)

In [ ]:
%timeit traj_spl.get_control()

In [ ]:
%timeit traj_spl.dvdt(0.1,y0[:2],y0[2:],state)

In [ ]:
%timeit traj_spl.timestep(0.1, y0)

In [ ]:
y = jnp.hstack([x0, y, operating_floor])


In [ ]:
y

In [ ]:
# Compare 8.16 µs to   45.7 ms
# 45.7 ms / 8.16 µs = 5600
45.7e-3 / 8.16e-6

In [ ]:
from diffrax import diffeqsolve, ODETerm, SaveAt, Dopri5, PIDController

term = ODETerm(traj_spl.timestep)
solver = Dopri5()
ts = jnp.linspace(0, 100, 101)
saveat = SaveAt(ts=ts)
stepsize_controller = PIDController(rtol=1e-5, atol=1e-5)

solution = diffeqsolve(term, solver, t0=0., t1=ts[-1], dt0=0.0001, y0=y0, saveat=saveat,
                    stepsize_controller=stepsize_controller, throw=False)

In [ ]:
solution.ys

In [ ]:
l = jax.grad(traj_spl.dvdt)

# Get the signature of lambda function
import inspect
inspect.signature(l).parameters
l( 
    jnp.array(0.1), 
    jnp.array([0., 0.]), 
    jnp.array([20., 0.]), 
    prev_state= jnp.array([0., 0., 0., 0.,0., 0.,0., 0.])
)

In [ ]:
import jax.numpy as jnp

from ICARUS.mission.trajectory.integrate import RK4systems
# velocity is in the same direction as the derivative of the polynomial
g1 = jnp.arctan(
    traj_spl.dy_dx(x0[0])
)
v0_mag = 20.
v0 = jnp.array([jnp.cos(g1), jnp.sin(g1)]) * v0_mag

dt = 0.01
tend = 200
t, xs, vs, states = RK4systems(t0, tend, dt, x0, v0, traj_spl )


In [ ]:
traj_spl.clear_history()
for statei, xi, vi, ti in zip(states, xs, vs, t):
    traj_spl.record_state(ti, xi, vi, *statei)
traj_spl.plot_history()

# Optimization

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Qt5Agg')


# fig = plt.figure()
# axs = fig.subplots(3, 2, squeeze=False).flatten().tolist()
# # plt.show(block=True)

def objective_goal_splines(y, dt = 0.05):
    x =  jnp.linspace(0, 10000, len(y)+2)
    y = jnp.hstack([x0[1], y, operating_floor])

    spline_i, title = CubicSpline_factory(x,y)

    trajectory = MissionTrajectory(
        title, 
        spline_i, 
        vehicle=mission_plane,
        verbosity= 2,
        operating_floor= 12.4
    )
    gamma = jnp.arctan(trajectory.dy_dx(x0[0]))
    v0 = jnp.array([np.cos(gamma), np.sin(gamma)]) * v0_mag


    t, xs, vs, _  = RK4systems(t0, tend, dt, x0, v0, trajectory)
    # trajectory.plot_history(axs,fig)
    # x = [x[0] for x in xs]
    # Remove nan values
    x = xs[:, 0]
    x = x[~jnp.isnan(x)]
    print(f"Max x: {jnp.max(x)}")
    return - jnp.max(x)

In [ ]:
from scipy.optimize import minimize
 
res_splines = minimize(
    objective_goal_splines,
    x0 = np.array([20, 20 , 20, 20, 20, 20,20, 20, 20,20, 20, 20,]),
    method='COBYLA',
    options={'disp': True, 'maxiter' : 3000},
)



In [ ]:
print(res_splines.x)
x0 = np.array([20, 20 , 20, 20, 20, 20,20, 20, 20,20, 20, 20,]),

In [ ]:
y  = res_splines.x
x = jnp.linspace(0, 10000, len(y)+2)
y = jnp.hstack([x0[1], y, operating_floor])

spline_best, title = CubicSpline_factory(x,y)
trajectory_best =  MissionTrajectory(
        title, 
        spline_best, 
        vehicle=mission_plane,
        verbosity= 2,
        operating_floor= 12.4
    )
gamma = jnp.arctan( trajectory_best.dy_dx(x0[0]) )
v0 = jnp.array([np.cos(gamma), np.sin(gamma)]) * v0_mag

t, xs, vs, states = RK4systems(t0, tend, dt, x0, v0, trajectory_best)

for statei, xi, vi, ti in zip(states, xs, vs, t):
    trajectory_best.record_state(ti, xi, vi, *statei)
# Plot Trajectory
trajectory_best.clear_history()
for statei, xi, vi, ti in zip(states, xs, vs, t):
    trajectory_best.record_state(ti, xi, vi, *statei)

trajectory_best.plot_history()